# Layerwise Sequential Unit Variance (LSUV)

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_07 import *

## 1. Get the MNIST data and prepare to build a CNN

[Jump_to lesson 11 video](https://course.fast.ai/videos/?lesson=11&t=235)

In [3]:
# get training and validation data
x_train,y_train,x_valid,y_valid = get_data()

# normalize validation data to training data
x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

# number of hidden layers, batch size
# note -- we don't need nh for this notebook
nh,bs = 50,512


# number of digit classes 
c = y_train.max().item()+1

# specify loss function
loss_func = F.cross_entropy

# create a DataBunch
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [4]:
# callback function to resize flattened image back into a 2D array
mnist_view = view_tfm(1,28,28)

# list of callback functions
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [5]:
# list number of channels for each desired convolution layer
nfs = [8,16,32,64,64]

## 2. Review documentation from the fastai library 

In [10]:
dir(cbfs[0])

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_order',
 'after_batch',
 'begin_fit',
 'name',
 'plot',
 'plot_loss',
 'plot_lr',
 'set_runner']

## 3. Build  a Baseline CNN model

In [11]:
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)
    
    def forward(self, x): return self.relu(self.conv(x))
    
    @property
    def bias(self): return -self.relu.sub
    @bias.setter
    def bias(self,v): self.relu.sub = -v
    @property
    def weight(self): return self.conv.weight

In [12]:
# set up for training
learn,run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

In [13]:
learn.model

Sequential(
  (0): ConvLayer(
    (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (relu): GeneralRelu()
  )
  (1): ConvLayer(
    (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (2): ConvLayer(
    (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (3): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (4): ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (5): AdaptiveAvgPool2d(output_size=1)
  (6): Lambda()
  (7): Linear(in_features=64, out_features=10, bias=True)
)

Now we're going to look at the paper [All You Need is a Good Init](https://arxiv.org/pdf/1511.06422.pdf), which introduces *Layer-wise Sequential Unit-Variance* (*LSUV*). We initialize our neural net with the usual technique, then we pass a batch through the model and check the outputs of the linear and convolutional layers. We can then rescale the weights according to the actual variance we observe on the activations, and subtract the mean we observe from the initial bias. That way we will have activations that stay normalized.

We repeat this process until we are satisfied with the mean/variance we observe.

Let's start by looking at a baseline:

In [14]:
%%time
# train
run.fit(2, learn)

train: [1.4467434375, tensor(0.5062, device='cuda:0')]
valid: [0.24174697265625, tensor(0.9300, device='cuda:0')]
train: [0.220030859375, tensor(0.9349, device='cuda:0')]
valid: [0.10628349609375, tensor(0.9671, device='cuda:0')]
Wall time: 7.29 s


## 4. Layer-wise sequential Unit Variance (LSUV)
Now we recreate our model and we'll try again with LSUV. Hopefully, we'll get better results!

In [15]:
# set up for training again
learn,run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

### break from code to check some things

In [16]:
dir(learn)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'data',
 'loss_func',
 'model',
 'opt']

In [17]:
learn.model

Sequential(
  (0): ConvLayer(
    (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (relu): GeneralRelu()
  )
  (1): ConvLayer(
    (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (2): ConvLayer(
    (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (3): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (4): ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (5): AdaptiveAvgPool2d(output_size=1)
  (6): Lambda()
  (7): Linear(in_features=64, out_features=10, bias=True)
)

In [18]:
dir(run)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'all_batches',
 'avg_stats',
 'batch_transform_x',
 'cbs',
 'cuda',
 'data',
 'fit',
 'in_train',
 'loss_func',
 'model',
 'one_batch',
 'opt',
 'recorder',
 'stop']

In [25]:
run.cbs

In [26]:
run.avg_stats

In [27]:
run.batch_transform_x

In [28]:
run.fit

<bound method Runner.fit of <exp.nb_05b.Runner object at 0x0000026986FBF1D0>>

In [29]:
# why do run.model, run.data, run.loss_func,run.opt return AttributeError: 'Runner' object has no attribute 'learn'

In [30]:
run.model

AttributeError: 'Runner' object has no attribute 'learn'

In [31]:
run.data

AttributeError: 'Runner' object has no attribute 'learn'

In [32]:
run.loss_func

AttributeError: 'Runner' object has no attribute 'learn'

In [33]:
run.opt

AttributeError: 'Runner' object has no attribute 'learn'

In [34]:
run.all_batches

<bound method Runner.all_batches of <exp.nb_05b.Runner object at 0x0000026986FBF1D0>>

In [35]:
run.stop

False

In [36]:
cbfs[0].set_runner

<function exp.nb_05b.Callback.set_runner(self, run)>

In [39]:
class Runner():
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            cb = cbf()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs

    # Question: how does run have access to the Learner object?
    @property
    def opt(self):       return self.learn.opt
    @property
    def model(self):     return self.learn.model
    @property
    def loss_func(self): return self.learn.loss_func
    @property
    def data(self):      return self.learn.data

    def one_batch(self, xb, yb):
        try:
            self.xb,self.yb = xb,yb
            self('begin_batch')
            self.pred = self.model(self.xb)
            self('after_pred')
            self.loss = self.loss_func(self.pred, self.yb)
            self('after_loss')
            if not self.in_train: return
            self.loss.backward()
            self('after_backward')
            self.opt.step()
            self('after_step')
            self.opt.zero_grad()
        except CancelBatchException: self('after_cancel_batch')
        finally: self('after_batch')

    def all_batches(self, dl):
        self.iters = len(dl)
        try:
            for xb,yb in dl: self.one_batch(xb, yb)
        except CancelEpochException: self('after_cancel_epoch')

    def fit(self, epochs, learn):
        self.epochs,self.learn,self.loss = epochs,learn,tensor(0.) 

        try:
            for cb in self.cbs: cb.set_runner(self)
            self('begin_fit')
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.data.train_dl)

                with torch.no_grad():
                    if not self('begin_validate'): self.all_batches(self.data.valid_dl)
                self('after_epoch')

        except CancelTrainException: self('after_cancel_train')
        finally:
            self('after_fit')
            self.learn = None

    def __call__(self, cb_name):
        res = False
        # Question: won't the following line always return res = False?
        for cb in sorted(self.cbs, key=lambda x: x._order): res = cb(cb_name) and res
        return res

### continue with code
Helper function to get one batch of a given dataloader, with the callbacks called to preprocess it.

In [40]:
#export
def get_batch(dl, run):
    run.xb,run.yb = next(iter(dl))
    # associat each callback with current Runner object
    for cb in run.cbs: cb.set_runner(run)
    run('begin_batch')
    # return a batch
    return run.xb,run.yb

In [41]:
# get a batch
xb,yb = get_batch(data.train_dl, run)

In [42]:
print(xb.size())
print(xb[0,0,0,:])

torch.Size([512, 1, 28, 28])
tensor([-0.4245, -0.4245, -0.4245, -0.4245, -0.4245, -0.4245, -0.4245, -0.4245,
        -0.4245, -0.4245, -0.4245, -0.4245, -0.4245, -0.4245, -0.4245, -0.4245,
        -0.4245, -0.4245, -0.4245, -0.4245, -0.4245, -0.4245, -0.4245, -0.4245,
        -0.4245, -0.4245, -0.4245, -0.4245], device='cuda:0')


We only want the outputs of convolutional or linear layers. To find them, we need a recursive function. We can use `sum(list, [])` to concatenate the lists the function finds (`sum` applies the + operate between the elements of the list you pass it, beginning with the initial state in the second argument).

In [43]:
#export
def find_modules(m, cond):
    if cond(m): return [m]
    return sum([find_modules(o,cond) for o in m.children()], [])

def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
    return isinstance(l, lin_layers)

In [44]:
mods = find_modules(learn.model, lambda o: isinstance(o,ConvLayer))

In [45]:
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 )]

This is a helper function to grab the mean and std of the output of a hooked layer.

In [46]:
def append_stat(hook, mod, inp, outp):
    d = outp.data
    hook.mean,hook.std = d.mean().item(),d.std().item()

In [47]:
# put model on the GPU
mdl = learn.model.cuda()

## Break from code to look up some docs in the fastai library

In [48]:
dir(mdl)

['__call__',
 '__class__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backend',
 '_backward_hooks',
 '_buffers',
 '_construct',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_item_by_idx',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_tracing_name',
 '_version',
 'add_module',
 'apply',
 'buffers',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 '

##  resume with the code
So now we can look at the mean and std of the conv layers of our model.

In [71]:
# train a batch
with Hooks(mods, append_stat) as hooks:
    # forward pass
    mdl(xb)
    for hook in hooks: print(hook.mean,hook.std)

AttributeError: 'Hook' object has no attribute 'mean'

We first adjust the bias terms to make the means 0, then we adjust the standard deviations to make the stds 1 (with a threshold of 1e-3). The `mdl(xb) is not None` clause is just there to pass `xb` through `mdl` and compute all the activations so that the hooks get updated. 

In [32]:
learn.model[7].bias

Parameter containing:
tensor([-0.0373, -0.1247, -0.0927, -0.1047,  0.1005, -0.0499, -0.1082,  0.0650,
         0.0426, -0.0307], device='cuda:0', requires_grad=True)

### Note: switching the order, i.e. scaling weights by variance first, then the removing the mean from the bias achieves near-perfect normalization

In [41]:
#export
# LSUV algorithm
def lsuv_module(m, xb):
    h = Hook(m, append_stat)
    while mdl(xb) is not None and abs(h.std-1) > 1e-3: m.weight.data /= h.std
    while mdl(xb) is not None and abs(h.mean)  > 1e-3: m.bias -= h.mean

    h.remove()
    return h.mean,h.std

We execute that initialization on all the conv layers in order:

In [42]:
for m in mods: print(lsuv_module(m, xb))

(-1.0035475561664953e-08, 0.9999998807907104)
(1.1099844599016251e-08, 0.9999998807907104)
(-2.0139850676059723e-08, 1.0)
(8.149072527885437e-09, 1.0)
(-1.4901161193847656e-08, 0.9999999403953552)


Note that the mean doesn't exactly stay at 0. since we change the standard deviation after by scaling the weight.
Note from jcat: But if you modify the code to scale the standard deviations first, then correct the mean, you achieve near-perfect normalization.

Then training is beginning on better grounds, and we get impoved accuracy after 2 epochs compared to the initial run with LSUV

In [47]:
# train with LSUV
%time run.fit(2, learn)

train: [0.4852893359375, tensor(0.8433, device='cuda:0')]
valid: [0.18036103515625, tensor(0.9479, device='cuda:0')]
train: [0.112610205078125, tensor(0.9645, device='cuda:0')]
valid: [0.09935478515625, tensor(0.9698, device='cuda:0')]
Wall time: 2.83 s


LSUV is particularly useful for more complex and deeper architectures that are hard to initialize to get unit variance at the last layer.

## Export

In [ ]:
!python notebook2script.py 07a_lsuv.ipynb